## **Workflow: Prompt chaining**

Prompt chaining decomposes a task into a sequence of steps, where each LLM call processes the output of the previous one.

You can add programmatic checks (see "gate” in the diagram below) on any intermediate steps to ensure that the process is still on track.

![Prompt chaining](resources/prompt-chaining.jpg)

## **Use cases:**

- Generating Marketing copy, then translating it into a different language.
- Writing an outline of a document, checking that the outline meets certain criteria, then writing the document based on the outline.
- Using an LLM to clean and standardize raw data, then passing the cleaned data to another LLM for insights, summaries, or visualizations.
- Generating a set of detailed questions based on a topic with one LLM, then passing those questions to another LLM to produce well-researched answers.

In [ ]:
%pip install openai pydantic --upgrade

In [ ]:
from typing import List
from pydantic import BaseModel
from openai import OpenAI

In [3]:
import getpass
import os

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("OPENAI_API_KEY")

In [6]:
client = OpenAI()

In [7]:
class Story(BaseModel):
    story: str
    title: str
    author: str

class StoryPlot(BaseModel):
    plot: str

class StoryPlots(BaseModel):
    plots: List[StoryPlot]

In [8]:
def serial_chain_workflow(topic: str, num_stories: int) -> List[Story]:
    """Run a serial chain of LLM calls to address the `input_query"""

    stories_plot_prompt = f'''Generate {num_stories} plot ideas for stories about {topic}.
    These should be short and concise, and should be suitable for a children's book.
    Later on these plot ideas will be used to generate full stories.'''

    stories_plot_response = client.beta.chat.completions.parse(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": stories_plot_prompt}],
        temperature=0.5,
        max_tokens=1000,
        response_format=StoryPlots
    )

    if (stories_plot_response.choices[0].message.parsed.plots == []):
        raise ValueError("No plots generated")

    print("Plots:")
    print(stories_plot_response.choices[0].message.parsed.plots)

    # For each story plot, generate a full story
    stories = []
    for plot in stories_plot_response.choices[0].message.parsed.plots:
        story_prompt = f'''Generate a full story based on the following plot: {plot.plot}.
        The story should be suitable for a children's book. The title should not be within the story part of the response.
        If there is no author, then write "Unknown"'''

        story_response = client.beta.chat.completions.parse(
            model="gpt-4o",
            messages=[{"role": "user", "content": story_prompt}],
            temperature=0.5,
            max_tokens=1000,
            response_format=Story
        )

        if (story_response.choices[0].message.parsed.story == ""):
            raise ValueError("No story generated")

        stories.append(story_response.choices[0].message.parsed)
    return stories

In [12]:
stories = serial_chain_workflow("The senior dev, Levente", 2)

Plots:
[StoryPlot(plot='Levente, the wise senior developer, discovers that the code for a magical app has gone missing! With the help of his friends, a curious young coder named Mia and a playful robot named Byte, they embark on a thrilling adventure through the digital world to find the lost code. Along the way, they learn about teamwork, creativity, and the importance of sharing knowledge.'), StoryPlot(plot="When a mysterious bug starts causing chaos in the coding world, Levente the senior developer must rally his team of young programmers to solve the problem. Together, they build a 'Bug-Busting' machine that helps them understand the importance of debugging and perseverance. Through fun challenges and clever problem-solving, they turn the bug into a feature that helps everyone!")]


In [11]:
print(stories[0].title)
print(stories[0].author)

The Cloudy Conundrum
Unknown
